Copyright (c) Microsoft Corporation. All rights reserved.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/tutorials/regression-part2-automated-ml.png)

![alt text](https://www.microsoftevents.com/accounts/register123/microsoft/msft-v1/c-and-e-v2/events/ce2-ce-2c-mec0028133/Azure%20Academy%20banner_Data.png "Logo Title Text 1")

# HOL01: Azure Machine Learning serice

This lab guides you through Azure Machine Learning service - creation & setup, building experiment and train model (also use automated machine learning technique).

In this use case you build a regression model to predict NYC taxi fare prices. 
This process accepts training data and configuration settings, and automatically iterates through combinations of different feature normalization/standardization methods, models, and hyperparameter settings to arrive at the best model.

In this lab you learn the following tasks:

* Create Azure Machine Learning Workspace
* Download, transform, and clean data using Azure Open Datasets
* Train an automated machine learning regression model
* Calculate model accuracy

If you don’t have an Azure subscription, create a free account before you begin. Try the [free or paid version](https://aka.ms/AMLFree) of Azure Machine Learning service today.

## Prerequisites

* Existing AML Workspace - step can be found on GitHub repo for this lab [AzureAcademy-DataAnalyst-II-ML-AI-HOL01-AML.md](https://github.com/michalmar/azure-labs/blob/master/AzureAcademy-DataAnalyst-II-ML-AI-HOL01-AML.md)

* check and update VM environemt

`pip install --upgrade azureml-sdk[explain,automl,notebooks] azureml-opendatasets azureml-widgets "urllib3==1.24"`


# PART 1: Download and prepare data

We will use [Azure Open Datasets](https://docs.microsoft.com/en-us/azure/open-datasets/overview-what-are-open-datasets) - curated public datasets that you can use to add scenario-specific features to machine learning solutions for more accurate models. Open Datasets are in the cloud on Microsoft Azure and are readily available to Azure Databricks, Machine Learning service, and Machine Learning Studio. You can also access the datasets through APIs and use them in other products, such as Power BI and Azure Data Factory.


We will use particular dataset: [NYC Taxi & Limousine Commission - green taxi trip records](https://azure.microsoft.com/en-us/services/open-datasets/catalog/nyc-taxi-limousine-commission-green-taxi-trip-records/)



Import the necessary packages. The Open Datasets package contains a class representing each data source (`NycTlcGreen` for example) to easily filter date parameters before downloading.

In [ ]:
from azureml.opendatasets import NycTlcGreen
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

%config IPCompleter.greedy=True

Begin by creating a dataframe to hold the taxi data. When working in a non-Spark environment, Open Datasets only allows downloading one month of data at a time with certain classes to avoid `MemoryError` with large datasets. To download taxi data, iteratively fetch one month at a time, and before appending it to `green_taxi_df` randomly sample 2,000 records from each month to avoid bloating the dataframe. Then preview the data.

In [ ]:
# green_taxi_df = pd.read_csv("./data/taxi_raw_df.csv")
# green_taxi_df['lpepPickupDatetime'] = green_taxi_df['lpepPickupDatetime'].astype('datetime64[ns]')
# # green_taxi_df.head(10)

In [ ]:
green_taxi_df = pd.DataFrame([])
start = datetime.strptime("1/1/2015","%m/%d/%Y")
end = datetime.strptime("1/31/2015","%m/%d/%Y")

for sample_month in range(12):
    temp_df_green = NycTlcGreen(start + relativedelta(months=sample_month), end + relativedelta(months=sample_month)) \
        .to_pandas_dataframe()
    green_taxi_df = green_taxi_df.append(temp_df_green.sample(2000))
    
green_taxi_df.head(10)

In [ ]:
green_taxi_df.to_csv("./data/taxi_raw_df.csv", index=False)

Now that the initial data is loaded, define a function to create various time-based features from the pickup datetime field. This will create new fields for the month number, day of month, day of week, and hour of day, and will allow the model to factor in time-based seasonality. 

Use the `apply()` function on the dataframe to iteratively apply the `build_time_features()` function to each row in the taxi data.

In [ ]:
def build_time_features(vector):
    pickup_datetime = vector[0]
    month_num = pickup_datetime.month
    day_of_month = pickup_datetime.day
    day_of_week = pickup_datetime.weekday()
    hour_of_day = pickup_datetime.hour
    
    return pd.Series((month_num, day_of_month, day_of_week, hour_of_day))

green_taxi_df[["month_num", "day_of_month","day_of_week", "hour_of_day"]] = green_taxi_df[["lpepPickupDatetime"]].apply(build_time_features, axis=1)
green_taxi_df.head(10)

Remove some of the columns that you won't need for training or additional feature building.

In [ ]:
columns_to_remove = ["lpepPickupDatetime", "lpepDropoffDatetime", "puLocationId", "doLocationId", "extra", "mtaTax",
                     "improvementSurcharge", "tollsAmount", "ehailFee", "tripType", "rateCodeID", 
                     "storeAndFwdFlag", "paymentType", "fareAmount", "tipAmount"
                    ]
for col in columns_to_remove:
    green_taxi_df.pop(col)
    
green_taxi_df.head(5)

### Cleanse data 

Run the `describe()` function on the new dataframe to see summary statistics for each field.

In [ ]:
green_taxi_df.describe()

In [ ]:
# Will allow us to embed images in the notebook
%matplotlib inline
import matplotlib.pyplot as plt


In [ ]:
plt.rcParams['figure.figsize'] = [16, 10]
boxplot = green_taxi_df.boxplot(column=list(green_taxi_df.columns[1:-4]))

From the summary statistics, you see that there are several fields that have outliers or values that will reduce model accuracy. First filter the lat/long fields to be within the bounds of the Manhattan area. This will filter out longer taxi trips or trips that are outliers in respect to their relationship with other features. 

Additionally filter the `tripDistance` field to be greater than zero but less than 31 miles (the haversine distance between the two lat/long pairs). This eliminates long outlier trips that have inconsistent trip cost.

Lastly, the `totalAmount` field has negative values for the taxi fares, which don't make sense in the context of our model, and the `passengerCount` field has bad data with the minimum values being zero.

Filter out these anomalies using query functions, and then remove the last few columns unnecessary for training.

In [ ]:
final_df = green_taxi_df.query("pickupLatitude>=40.53 and pickupLatitude<=40.88")
final_df = final_df.query("pickupLongitude>=-74.09 and pickupLongitude<=-73.72")
final_df = final_df.query("tripDistance>=0.25 and tripDistance<31")
final_df = final_df.query("passengerCount>0 and totalAmount>0")

columns_to_remove_for_training = ["pickupLongitude", "pickupLatitude", "dropoffLongitude", "dropoffLatitude"]
for col in columns_to_remove_for_training:
    final_df.pop(col)

Call `describe()` again on the data to ensure cleansing worked as expected. You now have a prepared and cleansed set of taxi, holiday, and weather data to use for machine learning model training.

In [ ]:
final_df.describe()

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(final_df, test_size=0.2, random_state=223)

print(f'train:{len(list(train["vendorID"]))} \ntest: {len(list(test["vendorID"]))}')

final_df.to_csv("./data/taxi_final_df.csv", index=False)
train.to_csv("./data/taxi_final_df_train.csv", index=False)
test.to_csv("./data/taxi_final_df_test.csv", index=False)

## PART 2: Train within notebook regression model with AML Serice

In PART2 you train simple regression model within the notebook environement while logging metrics and output trhough AML service Experiment. Also you try to run single parameter sweep of the regression model.

* first you create configure a connection to workspace
* then run the simple training
* lastly you run simple parameter sweep of a regression model
* review results

In [ ]:
final_df = pd.read_csv("./data/taxi_final_df.csv")
# final_df.head(10)

### Configure workspace


Create a workspace object from the existing workspace. A [Workspace](https://docs.microsoft.com/python/api/azureml-core/azureml.core.workspace.workspace?view=azure-ml-py) is a class that accepts your Azure subscription and resource information. It also creates a cloud resource to monitor and track your model runs. `Workspace.from_config()` reads the file **config.json** and loads the authentication details into an object named `ws`. `ws` is used throughout the rest of the code in this tutorial.

In [ ]:
from azureml.core.workspace import Workspace
ws = Workspace.from_config()
print(f"Name: {ws.name}, Resource group: {ws.resource_group}, Location: {ws.location}, Subscription: {ws.subscription_id}")

### Train locally within notebook

Split the data into training and test sets by using the `train_test_split` function in the `scikit-learn` library. This function segregates the data into the x (**features**) data set for model training and the y (**values to predict**) data set for testing. The `test_size` parameter determines the percentage of data to allocate to testing. The `random_state` parameter sets a seed to the random generator, so that your train-test splits are deterministic.

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib

import lightgbm as lgb


y_df = final_df.pop("totalAmount")
x_df = final_df

x_train, x_test, y_train, y_test = train_test_split(x_df, y_df, test_size=0.2, random_state=223)

Let's use scikit-learn to train a simple LightGBM regression model.  We use AML to record interesting information about the model in an Experiment.  An Experiment contains a series of trials called Runs.  During this trial we use AML in the following way:
* We access an experiment from our AML workspace by name, which will be created if it doesn't exist
* We use `start_logging` to create a new run in this experiment
* We use `run.log()` to record a parameter, num_leaves, and an accuracy measure - the Mean Squared Error (MSE) to the run.  We will be able to review and compare these measures in the Azure Portal at a later time.
* We store the resulting model in the **outputs** directory, which is automatically captured by AML when the run is complete.
* We use `run.complete()` to indicate that the run is over and results can be captured and finalized

In [ ]:
from azureml.core import Experiment

# Get an experiment object from Azure Machine Learning
experiment = Experiment(workspace=ws, name="train-within-notebook-lightgbm")

# Create a run object in the experiment
run =  experiment.start_logging()


# Log the algorithm parameters to the run
run.log('num_leaves', 31)
run.log('learning_rate', 0.05)
run.log('n_estimators', 20)

# setup model, train and test
gbm = lgb.LGBMRegressor(num_leaves=31,
                        learning_rate=0.05,
                        n_estimators=20)
model_gbm = gbm.fit(x_train, y_train,
        eval_set=[(x_test, y_test)],
        eval_metric='l1',
        early_stopping_rounds=5)

preds = model_gbm.predict(x_test)

# Output the Mean Squared Error to the notebook and to the run
print('Mean Squared Error is', mean_squared_error(y_test, preds))
run.log('mse', mean_squared_error(y_test, preds))

# Save the model to the outputs directory for capture
model_file_name = './outputs/model.pkl'

joblib.dump(value = model_gbm, filename = model_file_name)

# upload the model file explicitly into artifacts 
run.upload_file(name = model_file_name, path_or_stream = model_file_name)

# Complete the run
run.complete()

### Simple parameter sweep
Now let's take the same concept from above and modify the **num_leaves** parameter.  For each value of num_leaves we will create a run that will store metrics and the resulting model.  In the end we can use the captured run history to determine which model was the best for us to deploy. 

Note that by using `with experiment.start_logging() as run` AML will automatically call `run.complete()` at the end of each loop.

This example also uses the **tqdm** library to provide a thermometer feedback

In [ ]:
import numpy as np
from tqdm import tqdm
# experiment = Experiment(workspace=ws, name="train-locally-within-notebook-sweep3")
# list of numbers from 0 to 1.0 with a 0.05 interval
num_leaves_sweep = np.arange(5, 35, 2)
mses = []

with experiment.start_logging() as run:

    for num_leaves in tqdm(num_leaves_sweep):
        # create a bunch of runs, each train a model with a different parameters
        with run.child_run() as child_run:
            gbm = lgb.LGBMRegressor(num_leaves=num_leaves,
                                    learning_rate=0.05,
                                    n_estimators=20,
                                    silent=True)
            model_gbm = gbm.fit(x_train, y_train,
                    eval_set=[(x_test, y_test)],
                    eval_metric='l1',
                    early_stopping_rounds=5
                    , verbose=False)

            preds = model_gbm.predict(x_test)
            mse = mean_squared_error(y_true=y_test, y_pred=preds)

            # log alpha, mean_squared_error and feature names in run history
            child_run.log(name="num_leaves", value=num_leaves)
            child_run.log(name="mse", value=mse)
            mses.append(mse)

    run.log_list(name="mses", value=mses, description='')


### Viewing run results
Azure Machine Learning stores all the details about the run in the Azure cloud.  Let's access those details by retrieving a link to the run using the default run output.  Clicking on the resulting link will take you to an interactive page presenting all run information.

In [ ]:
run

An experiment is a logical container in an Azure ML Workspace. It contains a series of trials called Runs. As such, it hosts run records such as run metrics, logs, and other output artifacts from your experiments.

The purpose of this step is to have data points to test the finished model that haven't been used to train the model, in order to measure true accuracy. 

In other words, a well-trained model should be able to accurately make predictions from data it hasn't already seen. You now have data prepared for auto-training a machine learning model.

## PART 3: Train Automated ML Regresion model on AML remote Compute

In PART 3, you train multiple Regression models at once and select the best one via Autmated ML componement of Azure Machine Learning service. This happens on remote AML compute - simple auto-scaled cluster of machines for parallel training.

The steps are:
* configure datasource - remote storage shared between the parallel runs
* configure AML compute target
* configure and run Automated ML Experiment
* review results

Get default blob store associated with your workspace. Alternatively, you can attach your own blob storage to the Workspace - see [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-access-data).

In [ ]:
ds = ws.datastores['workspaceblobstore']
# ds = ws.get_default_datastore()
for attr, value in ds.__dict__.items():
    if (attr in ['name', 'datastore_type', 'container_name', 'account_name']):
        print(f"{attr}: {value}")

Upload prepared data into associated Datastore.

In [ ]:
ds.upload(src_dir='./data', target_path='data', overwrite=True, show_progress=True)

### Configure Compute Target

Create Compute target in Portal - alternativelly you could create using Pyhton SDK.

#### Set up in Azure portal

You can access the compute targets that are associated with your workspace in the Azure portal.  You can use the portal to:

* [View  compute targets](#portal-view) attached to your workspace
* [Create a compute target](#portal-create) in your workspace
* [Attach a compute target](#portal-reuse) that was created outside the workspace

After a target is created and attached to your workspace, you will use it in your run configuration with a `ComputeTarget` object: 

```python
from azureml.core.compute import ComputeTarget
myvm = ComputeTarget(workspace=ws, name='my-vm-name')
```

#### <a id="portal-view"></a>View compute targets


To see the compute targets for your workspace, use the following steps:

1. Navigate to the [Azure portal](https://portal.azure.com) and open your workspace. 
1. Under __Applications__, select __Compute__.

    [![View compute tab](./media/how-to-set-up-training-targets/azure-machine-learning-service-workspace.png)](./media/how-to-set-up-training-targets/azure-machine-learning-service-workspace-expanded.png)

#### <a id="portal-create"></a>Create a compute target

Follow the previous steps to view the list of compute targets. Then use these steps to create a compute target: 

1. Select the plus sign (+) to add a compute target.

    ![Add a compute target](./media/how-to-set-up-training-targets/add-compute-target.png) 

1. Enter a name for the compute target. 

1. Select **Machine Learning Compute** as the type of compute to use for __Training__. 

    >[!NOTE]
    >Azure Machine Learning Compute is the only  managed-compute resource you can create in the Azure portal.  All other compute resources can be attached after they are created.

1. Fill out the form. Provide values for the required properties, especially **VM Family**, and the **maximum nodes** to use to spin up the compute. For this scenario use **maximum nodes=10**

1. Select __Create__.


1. View the status of the create operation by selecting the compute target from the list:

    ![Select a compute target to view the create operation status](./media/how-to-set-up-training-targets/View_list.png)

1. You then see the details for the compute target: 

    ![View the computer target details](./media/how-to-set-up-training-targets/compute-target-details.png) 



### Select the created compute target for our experiment

Reuse the name of the cluster compute you created in preview step and set appropriatelly variable:

```python
amlcompute_cluster_name = "<#Name your cluster#>" 
```

In [ ]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget

amlcompute_cluster_name = "azdemocluster" #Name your cluster
compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)

# Use the 'status' property to get a detailed status for the current cluster. 
cts = compute_target.status.serialize()
print(f'Found existing compute target: {amlcompute_cluster_name}\n({"cluster is running" if (int(cts["currentNodeCount"])>0) else "cluster is idle"}) currentNodeCount: {cts["currentNodeCount"]}, vmPriority: {cts["vmPriority"]}, vmSize: {cts["vmSize"]}')
   

### Configure Automated ML

Observe parameters of DataReferenceConfiguration:

* `path_on_datastore`...folder in container
* `path_on_compute`...folder where the data is mounted/downloaded
* `mode`...wheter download or just mount

In [ ]:
from azureml.core.runconfig import DataReferenceConfiguration

dr = DataReferenceConfiguration(datastore_name=ds.name, 
                   path_on_datastore='data', 
                   path_on_compute='/tmp/azureml_runs',
                   mode='download', # download files from datastore to compute target
                   overwrite=False)

The RunConfiguration sets the docker Python environment - packages and Conda dependencies.

In [ ]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
import pkg_resources

# create a new RunConfig object
conda_run_config = RunConfiguration(framework="python")

# Set compute target to the Linux DSVM
conda_run_config.target = compute_target
# set the data reference of the run coonfiguration
conda_run_config.data_references = {ds.name: dr}

pandas_dependency = 'pandas==' + pkg_resources.get_distribution("pandas").version

cd = CondaDependencies.create(pip_packages=['azureml-sdk[automl]'], conda_packages=['numpy',pandas_dependency])
conda_run_config.environment.python.conda_dependencies = cd

Project folder gets uploaded into docker and will be the ``working directory`` of the executed code.

In [ ]:
import os

project_folder = "aml_prj"

if not os.path.exists(project_folder):
    os.makedirs(project_folder)
else:
    print(f"folder '{project_folder}' aready there")

### Create `get_data.py` script

This scripts is simply used by individual run to get the data for particular run. Returns `X` variables and `y` (Target) variable.

In [ ]:
%%writefile $project_folder/get_data.py
import pandas as pd
def get_data():
    try:
        train = pd.read_csv("/tmp/azureml_runs/data/taxi_final_df_train.csv")
        # train = pd.read_csv("./taxi_final_df_test.csv")
        
        y_train = train.pop("totalAmount")
        X_train = train

        print(f'train:{len(list(train["vendorID"]))}')
        
    except:
        print("")
        
    return { "X" : X_train.values, "y" : y_train.values.flatten() }

# get_data()


### Automatically train a model

To automatically train a model, take the following steps:
1. Define settings for the experiment run. Attach your training data to the configuration, and modify settings that control the training process.
1. Submit the experiment for model tuning. After submitting the experiment, the process iterates through different machine learning algorithms and hyperparameter settings, adhering to your defined constraints. It chooses the best-fit model by optimizing an accuracy metric.

#### Define training settings

Define the experiment parameter and model settings for training. View the full list of [settings](https://docs.microsoft.com/azure/machine-learning/service/how-to-configure-auto-train). Submitting the experiment with these default settings will take approximately 5-10 min, but if you want a shorter run time, reduce the `iterations` parameter.


|Property| Value in this tutorial |Description|
|----|----|---|
|**iteration_timeout_minutes**|2|Time limit in minutes for each iteration. Reduce this value to decrease total runtime.|
|**iterations**|20|Number of iterations. In each iteration, a new machine learning model is trained with your data. This is the primary value that affects total run time.|
|**primary_metric**| spearman_correlation | Metric that you want to optimize. The best-fit model will be chosen based on this metric.|
|**preprocess**| True | By using **True**, the experiment can preprocess the input data (handling missing data, converting text to numeric, etc.)|
|**verbosity**| logging.INFO | Controls the level of logging.|
|**n_cross_validations**|5|Number of cross-validation splits to perform when validation data is not specified.|
|**max_concurrent_iterations**|10|Number of parallel runs - should according to cluster size.|


In [ ]:
import logging

automl_settings = {
    "name": "NYC_Experiment_AutoML",
    "iteration_timeout_minutes": 2,
    "iterations": 20,
    "primary_metric": 'spearman_correlation',
    "preprocess": True,
    "verbosity": logging.INFO,
    "max_concurrent_iterations": 10,
    "n_cross_validations": 5
}

Use your defined training settings as a `**kwargs` parameter to an `AutoMLConfig` object. Additionally, specify your training data and the type of model, which is `regression` in this case.

In [ ]:
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(task='regression',
                             debug_log='automl_errors.log',
                             path=project_folder,
                             run_configuration=conda_run_config, 
                             compute_target = compute_target,
                             data_script=project_folder + "/get_data.py",
                             **automl_settings,
                            )

Automated machine learning pre-processing steps (feature normalization, handling missing data, converting text to numeric, etc.) become part of the underlying model. When using the model for predictions, the same pre-processing steps applied during training are applied to your input data automatically.

#### Train the automatic regression model

Create an experiment object in your workspace. An experiment acts as a container for your individual runs. Pass the defined `automl_config` object to the experiment, and set the output to `True` to view progress during the run. 

After starting the experiment, the output shown updates live as the experiment runs. For each iteration, you see the model type, the run duration, and the training accuracy. The field `BEST` tracks the best running training score based on your metric type.

During the training / experiment running you can observe the result and changes in Azure Portal. Also you can view the state of the runs and results using Widget below in sub-section **Explore results**

In [ ]:
from azureml.core.experiment import Experiment
experiment=Experiment(ws, 'openhack_automl_remote')
remote_run = experiment.submit(automl_config, show_output=False)

### Explore the results

Explore the results of automatic training with a [Jupyter widget](https://docs.microsoft.com/python/api/azureml-widgets/azureml.widgets?view=azure-ml-py). The widget allows you to see a graph and table of all individual run iterations, along with training accuracy metrics and metadata. Additionally, you can filter on different accuracy metrics than your primary metric with the dropdown selector.

In [ ]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

In [ ]:
 raise Exception('### INTENDED STOP ### to wait for asynchronous AutoML Job')

### Review trained model & results

#### Retrieve the best model

Select the best model from your iterations. The `get_output` function returns the best run and the fitted model for the last fit invocation. By using the overloads on `get_output`, you can retrieve the best run and fitted model for any logged metric or a particular iteration.

In [ ]:
best_run, fitted_model = remote_run.get_output()
print("Best Run:")
print(best_run)

print("")
print("Fitted model:")
print(fitted_model)

#### Test the best model accuracy

Use the best model to run predictions on the test data set to predict taxi fares. The function `predict` uses the best model and predicts the values of y, **trip cost**, from the `x_test` data set. Print the first 10 predicted cost values from `y_predict`.


In [ ]:
test = pd.read_csv("./data/taxi_final_df_test.csv")

In [ ]:
test.head()

In [ ]:
y_test = test.pop("totalAmount")
x_test = test

In [ ]:
y_predict = fitted_model.predict(x_test.values)
print(y_predict[:10])

Calculate the `root mean squared error` of the results. Convert the `y_test` dataframe to a list to compare to the predicted values. The function `mean_squared_error` takes two arrays of values and calculates the average squared error between them. Taking the square root of the result gives an error in the same units as the y variable, **cost**. It indicates roughly how far the taxi fare predictions are from the actual fares.


In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

y_actual = y_test.values.flatten().tolist()
rmse = sqrt(mean_squared_error(y_actual, y_predict))
print(f"RMSE: {rmse}")

Run the following code to calculate mean absolute percent error (MAPE) by using the full `y_actual` and `y_predict` data sets. This metric calculates an absolute difference between each predicted and actual value and sums all the differences. Then it expresses that sum as a percent of the total of the actual values.


In [ ]:
sum_actuals = sum_errors = 0

for actual_val, predict_val in zip(y_actual, y_predict):
    abs_error = actual_val - predict_val
    if abs_error < 0:
        abs_error = abs_error * -1

    sum_errors = sum_errors + abs_error
    sum_actuals = sum_actuals + actual_val

mean_abs_percent_error = sum_errors / sum_actuals
print("Model MAPE:")
print(mean_abs_percent_error)
print()
print("Model Accuracy:")
print(1 - mean_abs_percent_error)

From the two prediction accuracy metrics, you see that the model is fairly good at predicting taxi fares from the data set's features, typically within +- $4.00, and approximately 15% error. 

The traditional machine learning model development process is highly resource-intensive, and requires significant domain knowledge and time investment to run and compare the results of dozens of models. Using automated machine learning is a great way to rapidly test many different models for your scenario.

## PART 4: Model Deployment [optional]

In this part of the lab, you use trained best model to create deployment as webservice in Azure Container Instance (ACI). 

The steps of deployment process are:

* Model registration
* Creatig scoring script (the heart of WS)
* Setup inference environment
* Deploy to ACI as WS

### Model registration

In [ ]:
import pickle
filename = os.path.join(project_folder,"automl_best_model.pkl")
pickle.dump(fitted_model, open(filename, 'wb'))

In [ ]:
from azureml.core.model import Model

model = Model.register(model_path = "./{}/automl_best_model.pkl".format(project_folder),
                       model_name = "automl_best_model.pkl",
                       tags = {'area': "NYC", 'type': "regression"},
                       description = "Azure Academy - NYC model",
                       workspace = ws)

#### Get Model

You can get the registered model anytime as this is accessible from within your workspace.

In [ ]:
from azureml.core.model import Model
import os

model = Model(workspace=ws, name="automl_best_model.pkl")

### Scoring Script

The scoring script is basically getting the model, reading the input data and running/scoring the model and present outputs. It is tipically in separate folder but for this lab we will use the same forlder as for training.

In [ ]:
import os
script_folder = os.path.join(project_folder)

In [ ]:
%%writefile $script_folder/score.py

import pickle
import json
import numpy as np
from sklearn.externals import joblib
from sklearn.linear_model import Ridge
from azureml.core.model import Model
import azureml.train.automl

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType


def init():
    global model
    # note here "automl_best_model.pkl" is the name of the model registered under
    model_path = Model.get_model_path('automl_best_model.pkl')
    # deserialize the model file back into a sklearn model
    model = joblib.load(model_path)


# create schema of incomming request and outgoing response
input_sample = np.array([2,1,0.93,6,20,5,22])
output_sample = np.array([0.93])

@input_schema('data', NumpyParameterType(input_sample))
@output_schema(NumpyParameterType(output_sample))

# main function representing the actual scoring phase
def run(data):
    try:
        result = model.predict(data.reshape(1, -1))
        # you can return any datatype as long as it is JSON-serializable
        return result.tolist()
    except Exception as e:
        error = str(e)
        return error

### Create Inference Configuration


To deploy a model to Azure Container Instances, create a **deployment configuration** that describes the compute resources needed. For example, number of cores and memory. You also need an **inference configuration**, which describes the environment needed to host the model and web service. For more information on creating the inference configuration, see [How and where to deploy models](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-deploy-and-where).

In [ ]:
%%writefile $script_folder/myenv.yml

name: project_environment
dependencies:
  - python=3.7
  - pip:
    - azureml-defaults
    - azureml-sdk[automl]
    - scikit-learn
    - numpy
    - inference-schema[numpy-support]

In [ ]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(runtime= "python",
                                   entry_script=script_folder+"/score.py",
                                   conda_file=script_folder+"/myenv.yml")



### Create Deployment Configuration

`deploy_configuration(cpu_cores=None, memory_gb=None, tags=None, properties=None, description=None, location=None, auth_enabled=None, ssl_enabled=None, enable_app_insights=None, ssl_cert_pem_file=None, ssl_key_pem_file=None, ssl_cname=None, dns_name_label=None)`

In [ ]:
from azureml.core.webservice import AciWebservice, Webservice
from azureml.exceptions import WebserviceException

In [ ]:
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

### Deploy

Final step is the actual deployment. Use deployment config and inference config and the name of the service.

> note: If you see error that serice already exists, simple use `service.delete()`

> note: This steps may take up to 15 mins
> * it needs to create an docker image from the model and the configuration
> * create ACI with the parameters
> * deploy the docker into ACI

In [ ]:
aci_service_name = 'azureacademynyc'

service = Model.deploy(ws, aci_service_name, [model], inference_config, deployment_config)

service.wait_for_deployment(True)
print(service.state)

In [ ]:
print(service.get_logs())

### Test the webservices - score in real-time

In [ ]:
sample_df = pd.read_csv("./data/taxi_final_df_test.csv").sample()
vals = sample_df[["vendorID","passengerCount","tripDistance","month_num","day_of_month","day_of_week","hour_of_day","totalAmount"]].values
sample_data = vals.tolist()[0][0:-1]
sample_target = vals.tolist()[0][-1]

sample_df

In [ ]:
import json

test_sample = json.dumps({'data': sample_data})

test_sample_encoded = bytes(test_sample,encoding = 'utf8')
prediction = service.run(input_data=test_sample_encoded)

print(f"Prediction: {prediction[0]}\nActuals: {sample_target}")


## Clean up resources

Do not complete this section if you plan on running other Azure Machine Learning service tutorials.

### Delete service

In [ ]:
service.delete()

### Stop the notebook VM

If you used a cloud notebook server, stop the VM when you are not using it to reduce cost.

1. In your workspace, select **Notebook VMs**.
1. From the list, select the VM.
1. Select **Stop**.
1. When you're ready to use the server again, select **Start**.

### Delete everything

If you don't plan to use the resources you created, delete them, so you don't incur any charges.

1. In the Azure portal, select **Resource groups** on the far left.
1. From the list, select the resource group you created.
1. Select **Delete resource group**.
1. Enter the resource group name. Then select **Delete**.

You can also keep the resource group but delete a single workspace. Display the workspace properties and select **Delete**.

## Next steps

In this machine learning lab, you did the following tasks:

> * Configured a workspace and prepared data for an experiment.
> * Trained by using aregression model locally &  with custom parameters.
> * Trained by using autmated ML aregression model on achine learning compute.
> * Explored and reviewed training results.
> * Deploy model to ACI and test the web service

Visit [docs](https://docs.microsoft.com/azure/machine-learning/service/) with Azure Machine Learning service documenation and tutorials.

Learn by examples and code at [AML GitHub](https://github.com/Azure/MachineLearningNotebooks)